# statistics

* 读入预处理好的 csv 文件 `processed_data.csv` 并给出统计信息。<br/>
* 将 unigrams 以及 bigrams 的频率分布写入 pickle 文件。

In [1]:
num_tweets, num_pos_tweets, num_neu_tweets, num_neg_tweets = 0, 0, 0, 0
num_mentions, max_mentions = 0, 0
num_emojis, num_pos_emojis, num_neg_emojis, max_emojis = 0, 0, 0, 0
num_urls, max_urls = 0, 0
num_unigrams, num_unique_unigrams, min_unigrams, max_unigrams = 0, 0, 1e6, 0
num_bigrams, num_unique_bigrams = 0, 0

all_unigrams = []
all_bigrams = []

In [2]:
import pandas as pd
import pickle
from nltk import FreqDist
from nltk.tokenize import WordPunctTokenizer
word_tokenizer = WordPunctTokenizer()

processed_file_path = './processed_data.csv'
all_data = pd.read_csv(processed_file_path)

In [3]:
num_tweets = len(all_data)

In [4]:
def analyze_tweet(tweet):
    result = dict()
    result['MENTIONS'] = tweet.count('userMentionToken')
    result['URLS'] = tweet.count('urlToken')
    result['POS_EMOS'] = tweet.count(':)')
    result['NEG_EMOS'] = tweet.count(':(')
    unigrams = word_tokenizer.tokenize(tweet)
    result['UNIGRAMS'] = len(unigrams)
    bigrams = get_bigrams(unigrams)
    result['BIGRAMS'] = len(bigrams)
    return result, unigrams, bigrams

In [ ]:
def get_bigrams(unigrams):
    bigrams = []
    for i in range(len(unigrams) - 1):
        bigrams.append((unigrams[i], unigrams[i+1]))
    return bigrams

In [ ]:
for i in range(num_tweets):
    row = all_data.iloc[i]
    if row.sentiment == 'positive':
        num_pos_tweets += 1
    elif row.sentiment == 'neutral':
        num_neu_tweets += 1
    elif row.sentiment == 'negative':
        num_neg_tweets += 1
    result, unigrams, bigrams = analyze_tweet(row.text)
    num_mentions += result['MENTIONS']
    max_mentions = max(max_mentions, result['MENTIONS'])
    num_pos_emojis += result['POS_EMOS']
    num_neg_emojis += result['NEG_EMOS']
    max_emojis = max(max_emojis, result['POS_EMOS'] + result['NEG_EMOS'])
    num_urls += result['URLS']
    max_urls = max(max_urls, result['URLS'])
    num_unigrams += result['UNIGRAMS']
    min_unigrams = min(min_unigrams, result['UNIGRAMS'])
    max_unigrams = max(max_unigrams, result['UNIGRAMS'])
    all_unigrams.extend(unigrams)
    num_bigrams += result['BIGRAMS']
    all_bigrams.extend(bigrams)

num_emojis = num_pos_emojis + num_neg_emojis
unique_unigrams = list(set(all_unigrams))
with open('./unique-unigrams.txt', 'w') as uut:
    uut.write('\n'.join(unique_unigrams))
num_unique_unigrams = len(unique_unigrams)
num_unique_bigrams = len(set(all_bigrams))

In [ ]:
print('\nCalculating frequency distribution')
# Unigrams
unigrams_freq_dist = FreqDist(all_unigrams)
with open('./unigrams-freqdist.pkl', 'wb') as uf_pkl:
    pickle.dump(unigrams_freq_dist, uf_pkl)
print('Saved unigrams-frequency distribution to unigrams-freqdist.pkl')
# Bigrams
bigrams_freq_dist = FreqDist(all_bigrams)
with open('./bigrams-freqdist.pkl', 'wb') as bf_pkl:
    pickle.dump(bigrams_freq_dist, bf_pkl)
print('Saved bigrams-frequency distribution to bigrams-freqdist.pkl')

In [ ]:
print('\nAnalysis Statistics')
print('Tweets => Total: {}, Positive: {}, Neutral: {}, Negative: {}'.format(num_tweets, 
                                                                            num_pos_tweets, 
                                                                            num_neu_tweets, 
                                                                            num_neg_tweets))
print('User Mentions => Total: {}, Avg: {:.4f}, Max: {}'.format(num_mentions,
                                                                num_mentions / float(num_tweets),
                                                                max_mentions))
print('URLs => Total: {}, Avg: {:.4f}, Max: {}'.format(num_urls, 
                                                       num_urls / float(num_tweets),
                                                       max_urls))
print('Emojis => Total: {}, Positive: {}, Negative: {}, Avg: {:.4f}, Max: {}'.format(num_emojis,
                                                                                     num_pos_emojis,
                                                                                     num_neg_emojis,
                                                                                     num_emojis / float(num_tweets),
                                                                                     max_emojis))
print('Unigrams => Total: {}, Unique: {}, Avg: {:.4f}, Max: {}, Min: {}'.format(num_tweets,
                                                                                num_unique_unigrams,
                                                                                num_unigrams / float(num_tweets),
                                                                                max_unigrams,
                                                                                min_unigrams))
print('Bigrams => Total: {}, Unique: {}, Avg: {:.4f}'.format(num_bigrams,
                                                             num_unique_bigrams,
                                                             num_bigrams / float(num_tweets)))